# ACT方策モデルの訓練

## 有用な情報ソース

- [Getting started real world robot][a-1]
- [LeRobot Notebooks][a-2]
- [ACT][a-3]

[a-1]: https://huggingface.co/docs/lerobot/main/en/getting_started_real_world_robot#replay-an-episode
[a-2]: https://huggingface.co/docs/lerobot/main/en/notebooks
[a-3]: https://github.com/hayatoshibahara/act

## 環境構築

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

# 再起動後、次のセルから実行

In [ ]:
try: 
    from google.colab import userdata
    HF_USER = userdata.get("HF_USER")
    HF_TOKEN = userdata.get("HF_TOKEN")
    WANDB_API_KEY = userdata.get("WANDB_API_KEY")
    !git clone https://github.com/huggingface/lerobot.git
    !conda install ffmpeg=7.1.1 -c conda-forge
    !cd lerobot && pip install -e .
except:
    from dotenv import load_dotenv
    import os
    load_dotenv()
    HF_USER = os.getenv("HF_USER")
    HF_TOKEN = os.getenv("HF_TOKEN")
    WANDB_API_KEY = os.getenv("WANDB_API_KEY")

assert HF_USER
assert HF_TOKEN
assert WANDB_API_KEY

import wandb
wandb.login(key=WANDB_API_KEY)

## セットアップ

In [ ]:
if False:
    # 1. マイコンボードをUSBで接続する
    # 2. find_portコマンドを実行
    !python -m lerobot.find_port
    # 3. USBを外して、エンターキーを押す
    # 4. 「/dev/tty.usbmodem5A7A0182121」のようなポートが出力されるので変数に保存する
    #   !export LEADER_PORT="/dev/tty.usbmodem5A7A0182121"
    #   !export FOLLOWER_PORT="/dev/tty.usbmodem5A7A0182121"

In [ ]:
# フォロワーアームのIDを設定する
!export FOLLOWER_ID="follower_arm_0"

# リーダーのIDを設定する
!export LEADER_ID="leader_arm_0"

In [ ]:
# カメラのIDを確認する

if False:
    !lerobot-find-cameras opencv

In [ ]:
# カメラを設定し遠隔操作できるかを確認する

if False:
    !export CAMERAS="{ images.wrist.top: {type: opencv, index_or_path: 0, width: 640, height: 480, fps: 30}, images.top: {type: opencv, index_or_path: 1, width: 640, height: 480, fps: 30}}"

    !python -m lerobot.teleoperate \
        --robot.type=so101_follower \
        --robot.port=$FOLLOWER_PORT \
        --robot.id=$FOLLOWER_ID \
        --robot.cameras=$CAMERAS \
        --teleop.type=so101_leader \
        --teleop.port=$LEADER_PORT \
        --teleop.id=$LEADER_ID \
        --display_data=true

In [ ]:
# 自分で作成したデータセットのリポジトリを設定

!export DATASET_REPO_ID=EngineerCafeJP/record-test-2025-08-23-20-48-00

In [ ]:
# ダウンロードしたデータセットのエピソードを再生する

if False:
    !python -m lerobot.replay \
        --robot.type=so101_follower \
        --robot.port=$FOLLOWER_PORT \
        --robot.id=$FOLLOWER_ID \
        --dataset.repo_id=$DATASET_REPO_ID \
        --dataset.episode=0

## 訓練

In [ ]:
# ACTポリシーで学習する
# 訓練設定はTrainPipelineConfigで確認できる
# 20000ステップの場合、Google ColabのA100で1時間20分程度かかる
# Out of Memoryエラーが発生した場合はバッチサイズを減らし再起動する（デフォルトは8）
# A100の場合はバッチサイズ8（24GB/40GB消費）、T4の場合はバッチサイズは4（12GB/16GB消費）あたりが限界

!python lerobot/src/lerobot/scripts/train.py \
    --dataset.repo_id=$DATASET_REPO_ID \
    --policy.type=act \
    --policy.device=cuda \
    --policy.push_to_hub=true \
    --policy.repo_id=$HF_USER/act-so101-test \
    --output_dir=act_output \
    --job_name=act_so101_test \
    --resume=false \
    --num_workers=0 \
    --batch_size=8 \
    --steps=20000 \
    --save_freq=2000 \
    --wandb.enable=true

In [ ]:
# 中断した訓練を再開する場合

if False:
    python lerobot/src/lerobot/scripts/train.py \
        --config_path=act_output/checkpoints/last/pretrained_model/train_config.json \
        --resume=true

In [ ]:
# 事前学習済みモデルを別のデータセットで追加学習する場合

if False:
    !python lerobot/src/lerobot/scripts/train.py \
        --dataset.repo_id=EngineerCafeJP/record-test-2025-08-26-21-21-00 \
        --policy.path=EngineerCafeJP/act-so101-test \
        --policy.push_to_hub=true \
        --policy.repo_id=EngineerCafeJP/act-so101-test-2025-08-26-21-21-00 \
        --output_dir=act_output \
        --job_name=act_so101_test_2025-08-26-21-21-00 \
        --resume=false \
        --num_workers=0 \
        --batch_size=8 \
        --steps=20000 \
        --save_freq=2000 \
        --wandb.enable=true

簡単なタスクの場合、20kぐらいで早期終了できる:

![](asset/trained.png)

## アップロード

Google Colabの訓練中に「ターミナル」を開くことで、モデルを非同期にアップロードできる

In [ ]:
# 最も新しいチェックポイントをアップロードする場合

if False:
    !hf upload \
        --token $HF_TOKEN \
        $HF_USER/act_so101_test \
        act_output/checkpoints/last/pretrained_model \
        .

In [ ]:
# 特定のチェックポイントをアップロードする場合

if False:
    !hf upload \
        --token $HF_TOKEN \
        $HF_USER/act_so101_test_2000 \
        act_output/checkpoints/2000/pretrained_model \
        .

## 評価

訓練データのバリエーションが少ない場合、カメラの位置や向きに注意する

In [ ]:
# アップロードしたポリシーを使ってロボットを動かす
# 10エピソード分評価データを記録し、eval_*というデータセットを作成する

if False:
    !export EVAL_DATASET_REPO_ID=EngineerCafeJP/eval_test-2025-08-26-16-51
    !python -m lerobot.record  \
        --robot.type=so101_follower \
        --robot.port=$FOLLOWER_PORT \
        --robot.cameras=$CAMERAS \
        --robot.id=$FOLLOWER_ID \
        --display_data=false \
        --dataset.repo_id=$EVAL_DATASET_REPO_ID \
        --dataset.single_task="Grab box and put it in the yellow box" \
        --policy.path=$HF_USER/act-so101-test

# dataset.repo_idはeval_から始まる必要がある
# policy.pathはディレクトリでも良い:
# --policy.path=output/train/act_so101_test/checkpoints/last/pretrained_model